### Added new functions
1. Add standardized chrome selenium activation --> The best headache saver ever!
1. (To Do) Add Pysheets to make maintanence easier!

In [1]:
import sys
sys.path.insert(0, '/Users/cliff.chew/Downloads/self/python_functions')
from web_scraping import activate_chrome_selenium

In [2]:
from googleapiclient import discovery
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
import numpy as np
import itertools
import pygsheets
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [3]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
browser = activate_chrome_selenium()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/cliff.chew/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


### Log in first! 

In [5]:
info = pd.read_csv('nlb_credentials.csv')
account_name = info['values'][0]
password = info['values'][1]

login_page = 'https://cassamv2.nlb.gov.sg/cas/login'
browser.get(login_page)

# Actions on login page
username_tag = """//*[@id="username"]"""
element = browser.find_element_by_xpath(username_tag)
element.send_keys("{}".format(account_name))

password_tag = """//*[@id="password"]"""
element = browser.find_element_by_xpath(password_tag)
element.send_keys("{}".format(password))

login_button_2 = """//*[@id="fm1"]/section[4]/input[4]"""
browser.find_element_by_xpath(login_button_2).click()

### Loop through the pages! 

In [6]:
loans_link = "https://www.nlb.gov.sg/mylibrary/Loans"
browser.get(loans_link)

element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH , """//*[@id="current-loans"]/div[3]/table""")))

In [7]:
soup = bs(browser.page_source, "html5lib")

In [8]:
table_col = list()
for table in soup.find_all("table", class_="table table-bordered table-striped table-list"):
    for row in table.find_all('th'):
        table_col.append(row.text)
table_col = table_col[:5]

In [26]:
table_cells = list()
for table in soup.find_all("table", class_="table table-bordered table-striped table-list"):
    for row in table.find_all('td'):
        table_cells.append(row.text)

In [31]:
books = pd.DataFrame(np.array(table_cells).reshape(int(len(table_cells)/5), 5))
books.columns = ['no', 'title', 'code', 'due', 'renewed']
books = books[['title', 'code', 'due']]
books = books[1:].reset_index(drop=True)

books['title'] = [i.replace("Title: ", "").strip() for i in books['title']]
books['code'] = [i.replace("Barcode: ", "").strip() for i in books['code']]
books['due'] = [i.replace("Due on ", "") .strip() for i in books['due']]

### Authenticate into Google Drive 

In [32]:
google_auth = "/Users/cliff.chew/Downloads/self/python_functions/API Project-8b57e830c88b.json"
gc = pygsheets.authorize(service_file=google_auth)
name_of_worksheet = 'NLB Project'
sh = gc.open(name_of_worksheet)
wks = sh.worksheet_by_title("Current_borrowed")
wks.set_dataframe(books,(1,1))

wks.update_value('D1', 'Days Left')
wks.update_value('D2', '=ARRAYFORMULA(DAYS(C2:C{},$E$1))'.format(books.shape[0] + 1))

link to page: https://docs.google.com/spreadsheets/d/1s5oYU59jyU_QO3IIhCClyWGoC_MpW9L_h4l4djDUKO0/edit#gid=1021888748